In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pandas.util._doctools as doctools

p = doctools.TablePlotter()


# Merge, join, concatenate and compare

pandas provides various facilities for easily combining together Series or
DataFrame with various kinds of set logic for the indexes
and relational algebra functionality in the case of join / merge-type
operations.

In addition, pandas also provides utilities to compare two Series or DataFrame
and summarize their differences.


<a id='merging-concat'></a>

## Concatenating objects

The `concat()` function (in the main pandas namespace) does all of
the heavy lifting of performing concatenation operations along an axis while
performing optional set logic (union or intersection) of the indexes (if any) on
the other axes. Note that I say “if any” because there is only a single possible
axis of concatenation for Series.

Before diving into all of the details of `concat` and what it can do, here is
a simple example:

In [ ]:
df1 = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    },
    index=[0, 1, 2, 3],
)

df2 = pd.DataFrame(
    {
        "A": ["A4", "A5", "A6", "A7"],
        "B": ["B4", "B5", "B6", "B7"],
        "C": ["C4", "C5", "C6", "C7"],
        "D": ["D4", "D5", "D6", "D7"],
    },
    index=[4, 5, 6, 7],
)

df3 = pd.DataFrame(
    {
        "A": ["A8", "A9", "A10", "A11"],
        "B": ["B8", "B9", "B10", "B11"],
        "C": ["C8", "C9", "C10", "C11"],
        "D": ["D8", "D9", "D10", "D11"],
    },
    index=[8, 9, 10, 11],
)

frames = [df1, df2, df3]
result = pd.concat(frames)

In [ ]:
p.plot(frames, result, labels=["d1","d2","d3"], vertical=True);

Like its sibling function on ndarrays, `numpy.concatenate`, `pandas.concat`
takes a list or dict of homogeneously-typed objects and concatenates them with
some configurable handling of “what to do with the other axes”:

In [ ]:
pd.concat(
    [df1, df2, df3],   #objs,
    axis=0,
    join="outer",
    ignore_index=False,
    keys=None,
    levels=None,
    names=None,
    verify_integrity=False,
    copy=True,
)

- `objs` : a sequence or mapping of Series or DataFrame objects. If a
  dict is passed, the sorted keys will be used as the `keys` argument, unless
  it is passed, in which case the values will be selected (see below). Any None
  objects will be dropped silently unless they are all None in which case a
  ValueError will be raised.  
- `axis` : {0, 1, …}, default 0. The axis to concatenate along.  
- `join` : {‘inner’, ‘outer’}, default ‘outer’. How to handle indexes on
  other axis(es). Outer for union and inner for intersection.  
- `ignore_index` : boolean, default False. If True, do not use the index
  values on the concatenation axis. The resulting axis will be labeled 0, …,
  n - 1. This is useful if you are concatenating objects where the
  concatenation axis does not have meaningful indexing information. Note
  the index values on the other axes are still respected in the join.  
- `keys` : sequence, default None. Construct hierarchical index using the
  passed keys as the outermost level. If multiple levels passed, should
  contain tuples.  
- `levels` : list of sequences, default None. Specific levels (unique values)
  to use for constructing a MultiIndex. Otherwise they will be inferred from the
  keys.  
- `names` : list, default None. Names for the levels in the resulting
  hierarchical index.  
- `verify_integrity` : boolean, default False. Check whether the new
  concatenated axis contains duplicates. This can be very expensive relative
  to the actual data concatenation.  
- `copy` : boolean, default True. If False, do not copy data unnecessarily.  


Without a little bit of context many of these arguments don’t make much sense.
Let’s revisit the above example. Suppose we wanted to associate specific keys
with each of the pieces of the chopped up DataFrame. We can do this using the
`keys` argument:

In [ ]:
result = pd.concat(frames, keys=["x", "y", "z"])

In [ ]:
p.plot(frames, result, labels=["d1","d2","d3"], vertical=True);

As you can see (if you’ve read the rest of the documentation), the resulting
object’s index has a hierarchical index. This
means that we can now select out each chunk by key:

In [ ]:
result.loc["y"]

It’s not a stretch to see how this can be very useful. More detail on this
functionality below.

>**Note**
>
>It is worth noting that `concat()` (and therefore
`append()`) makes a full copy of the data, and that constantly
reusing this function can create a significant performance hit. If you need
to use the operation over several datasets, use a list comprehension.

```python
frames = [ process_your_file(f) for f in files ]
result = pd.concat(frames)
```


>**Note**
>
>When concatenating DataFrames with named axes, pandas will attempt to preserve
these index/column names whenever possible. In the case where all inputs share a
common name, this name will be assigned to the result. When the input names do
not all agree, the result will be unnamed. The same is true for `MultiIndex`,
but the logic is applied separately on a level-by-level basis.

### Set logic on the other axes

When gluing together multiple DataFrames, you have a choice of how to handle
the other axes (other than the one being concatenated). This can be done in
the following two ways:

- Take the union of them all, `join='outer'`. This is the default
  option as it results in zero information loss.  
- Take the intersection, `join='inner'`.  


Here is an example of each of these methods. First, the default `join='outer'`
behavior:

In [ ]:
df4 = pd.DataFrame(
    {
        "B": ["B2", "B3", "B6", "B7"],
        "D": ["D2", "D3", "D6", "D7"],
        "F": ["F2", "F3", "F6", "F7"],
    },
    index=[2, 3, 6, 7],
)
result = pd.concat([df1, df4], axis=1)

In [ ]:
p.plot([df1, df4], result, labels=["df1", "df4"], vertical=False);

Here is the same thing with `join='inner'`:

In [ ]:
result = pd.concat([df1, df4], axis=1, join="inner")

In [ ]:
p.plot([df1, df4], result, labels=["df1", "df4"], vertical=False);


Lastly, suppose we just wanted to reuse the *exact index* from the original
DataFrame:

In [ ]:
result = pd.concat([df1, df4], axis=1).reindex(df1.index)

Similarly, we could index before the concatenation:

In [ ]:
pd.concat([df1, df4.reindex(df1.index)], axis=1)

In [ ]:
p.plot([df1, df4], result, labels=["df1", "df4"], vertical=False);

<a id='merging-concatenation'></a>

### Concatenating using `append`

A useful shortcut to `concat()` are the `append()`
instance methods on `Series` and `DataFrame`. These methods actually predated
`concat`. They concatenate along `axis=0`, namely the index:

In [ ]:
result = df1.append(df2)

In [ ]:
p.plot([df1, df2], result, labels=["df1","df2"], vertical=True);

In the case of `DataFrame`, the indexes must be disjoint but the columns do not
need to be:

In [ ]:
result = df1.append(df4, sort=False)

In [ ]:
p.plot([df1, df4], result, labels=["df1","df4"], vertical=True);

`append` may take multiple objects to concatenate:

In [ ]:
result = df1.append([df2, df3])

In [ ]:
p.plot([df1, df2, df3], result, labels=["df1","df2","df3"], vertical=True);

>**Note**
>
>Unlike the `append()` method, which appends to the original list
and returns `None`, `append()`  here **does not** modify
`df1` and returns its copy with `df2` appended.


<a id='merging-ignore-index'></a>

### Ignoring indexes on the concatenation axis

For `DataFrame` objects which don’t have a meaningful index, you may wish
to append them and ignore the fact that they may have overlapping indexes. To
do this, use the `ignore_index` argument:

In [ ]:
result = pd.concat([df1, df4], ignore_index=True, sort=False)

In [ ]:
p.plot([df1, df4], result, labels=["df1","df4"], vertical=True);


This is also a valid argument to `DataFrame.append()`:

In [ ]:
result = df1.append(df4, ignore_index=True, sort=False)

In [ ]:
p.plot([df1, df4], result, labels=["df1","df4"], vertical=True);

### Concatenating with mixed ndims

You can concatenate a mix of `Series` and `DataFrame` objects. The
`Series` will be transformed to `DataFrame` with the column name as
the name of the `Series`.

In [ ]:
s1 = pd.Series(["X0", "X1", "X2", "X3"], name="X")
result = pd.concat([df1, s1], axis=1)

In [ ]:
p.plot([df1, s1], result, labels=["df1","s1"], vertical=False);

>**Note**
>
>Since we’re concatenating a `Series` to a `DataFrame`, we could have
achieved the same result with `DataFrame.assign()`. To concatenate an
arbitrary number of pandas objects (`DataFrame` or `Series`), use
`concat`.

If unnamed `Series` are passed they will be numbered consecutively.

In [ ]:
s2 = pd.Series(["_0", "_1", "_2", "_3"])
result = pd.concat([df1, s2, s2, s2], axis=1)

In [ ]:
p.plot([df1, s2], result, labels=["df1","s2"], vertical=False);



Passing `ignore_index=True` will drop all name references.

In [ ]:
result = pd.concat([df1, s1], axis=1, ignore_index=True)

In [ ]:
p.plot([df1, s1], result, labels=["df1","s1"], vertical=False);

### More concatenating with group keys

A fairly common use of the `keys` argument is to override the column names
when creating a new `DataFrame` based on existing `Series`.
Notice how the default behaviour consists on letting the resulting `DataFrame`
inherit the parent `Series`’ name, when these existed.

In [ ]:
s3 = pd.Series([0, 1, 2, 3], name="foo")
s4 = pd.Series([0, 1, 2, 3])
s5 = pd.Series([0, 1, 4, 5])

pd.concat([s3, s4, s5], axis=1)

Through the `keys` argument we can override the existing column names.

In [ ]:
pd.concat([s3, s4, s5], axis=1, keys=["red", "blue", "yellow"])

Let’s consider a variation of the very first example presented:

In [ ]:
result = pd.concat(frames, keys=["x", "y", "z"])

In [ ]:
p.plot([df1, df2, df3], result, labels=["df1","df2","df3"], vertical=False);

You can also pass a dict to `concat` in which case the dict keys will be used
for the `keys` argument (unless other keys are specified):

In [ ]:
pieces = {"x": df1, "y": df2, "z": df3}
result = pd.concat(pieces)

In [ ]:
p.plot([df1, df2, df3], result, labels=["df1","df2","df3"], vertical=False);

In [ ]:
result = pd.concat(pieces, keys=["z", "y"])

In [ ]:
p.plot([df1, df2, df3], result, labels=["df1","df2","df3"], vertical=False);

The MultiIndex created has levels that are constructed from the passed keys and
the index of the `DataFrame` pieces:

In [ ]:
result.index.levels

If you wish to specify other levels (as will occasionally be the case), you can
do so using the `levels` argument:

In [ ]:
result = pd.concat(
    pieces, keys=["x", "y", "z"], levels=[["z", "y", "x", "w"]], names=["group_key"]
)

In [ ]:
p.plot([df1, df2, df3], result, labels=["df1","df2","df3"], vertical=False);

In [ ]:
result.index.levels

This is fairly esoteric, but it is actually necessary for implementing things
like GroupBy where the order of a categorical variable is meaningful.


<a id='merging-append-row'></a>

### Appending rows to a DataFrame

While not especially efficient (since a new object must be created), you can
append a single row to a `DataFrame` by passing a `Series` or dict to
`append`, which returns a new `DataFrame` as above.

In [ ]:
s2 = pd.Series(["X0", "X1", "X2", "X3"], index=["A", "B", "C", "D"])
result = df1.append(s2, ignore_index=True)

In [ ]:
p.plot([df1, s2], result, labels=["df1","s2"], vertical=True);

You should use `ignore_index` with this method to instruct DataFrame to
discard its index. If you wish to preserve the index, you should construct an
appropriately-indexed DataFrame and append or concatenate those objects.

You can also pass a list of dicts or Series:

In [ ]:
dicts = [{"A": 1, "B": 2, "C": 3, "X": 4}, {"A": 5, "B": 6, "C": 7, "Y": 8}]
result = df1.append(dicts, ignore_index=True, sort=False)

In [ ]:
p.plot([df1, pd.DataFrame(dicts)], result, labels=["df1","dicts"], vertical=True);

## Database-style DataFrame or named Series joining/merging

pandas has full-featured, **high performance** in-memory join operations
idiomatically very similar to relational databases like SQL. These methods
perform significantly better (in some cases well over an order of magnitude
better) than other open source implementations (like `base::merge.data.frame`
in R). The reason for this is careful algorithmic design and the internal layout
of the data in `DataFrame`.

See the [cookbook](44_cookbook.ipynb#cookbook-merge) for some advanced strategies.

Users who are familiar with SQL but new to pandas might be interested in a
[comparison with SQL](../01_comparison/comparison_with_sql.ipynb#compare-with-sql-join).

pandas provides a single function, `merge()`, as the entry point for
all standard database join operations between `DataFrame` or named `Series` objects:

```python
pd.merge(
    left,
    right,
    how="inner",
    on=None,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)
```

- `left`: A DataFrame or named Series object.  
- `right`: Another DataFrame or named Series object.  
- `on`: Column or index level names to join on. Must be found in both the left
  and right DataFrame and/or Series objects. If not passed and `left_index` and
  `right_index` are `False`, the intersection of the columns in the
  DataFrames and/or Series will be inferred to be the join keys.  
- `left_on`: Columns or index levels from the left DataFrame or Series to use as
  keys. Can either be column names, index level names, or arrays with length
  equal to the length of the DataFrame or Series.  
- `right_on`: Columns or index levels from the right DataFrame or Series to use as
  keys. Can either be column names, index level names, or arrays with length
  equal to the length of the DataFrame or Series.  
- `left_index`: If `True`, use the index (row labels) from the left
  DataFrame or Series as its join key(s). In the case of a DataFrame or Series with a MultiIndex
  (hierarchical), the number of levels must match the number of join keys
  from the right DataFrame or Series.  
- `right_index`: Same usage as `left_index` for the right DataFrame or Series  
- `how`: One of `'left'`, `'right'`, `'outer'`, `'inner'`. Defaults
  to `inner`. See below for more detailed description of each method.  
- `sort`: Sort the result DataFrame by the join keys in lexicographical
  order. Defaults to `True`, setting to `False` will improve performance
  substantially in many cases.  
- `suffixes`: A tuple of string suffixes to apply to overlapping
  columns. Defaults to `('_x', '_y')`.  
- `copy`: Always copy data (default `True`) from the passed DataFrame or named Series
  objects, even when reindexing is not necessary. Cannot be avoided in many
  cases but may improve performance / memory usage. The cases where copying
  can be avoided are somewhat pathological but this option is provided
  nonetheless.  
- `indicator`: Add a column to the output DataFrame called `_merge`
  with information on the source of each row. `_merge` is Categorical-type
  and takes on a value of `left_only` for observations whose merge key
  only appears in `'left'` DataFrame or Series, `right_only` for observations whose
  merge key only appears in `'right'` DataFrame or Series, and `both` if the
  observation’s merge key is found in both.  
- `validate` : string, default None.
  If specified, checks if merge is of specified type.  
  
  - “one_to_one” or “1:1”: checks if merge keys are unique in both
    left and right datasets.  
  - “one_to_many” or “1:m”: checks if merge keys are unique in left
    dataset.  
  - “many_to_one” or “m:1”: checks if merge keys are unique in right
    dataset.  
  - “many_to_many” or “m:m”: allowed, but does not result in checks.  
  


>**Note**
>
>Support for specifying index levels as the `on`, `left_on`, and
`right_on` parameters was added in version 0.23.0.
Support for merging named `Series` objects was added in version 0.24.0.

The return type will be the same as `left`. If `left` is a `DataFrame` or named `Series`
and `right` is a subclass of `DataFrame`, the return type will still be `DataFrame`.

`merge` is a function in the pandas namespace, and it is also available as a
`DataFrame` instance method `merge()`, with the calling
`DataFrame` being implicitly considered the left object in the join.

The related `join()` method, uses `merge` internally for the
index-on-index (by default) and column(s)-on-index join. If you are joining on
index only, you may wish to use `DataFrame.join` to save yourself some typing.

### Brief primer on merge methods (relational algebra)

Experienced users of relational databases like SQL will be familiar with the
terminology used to describe join operations between two SQL-table like
structures (`DataFrame` objects). There are several cases to consider which
are very important to understand:

- **one-to-one** joins: for example when joining two `DataFrame` objects on
  their indexes (which must contain unique values).  
- **many-to-one** joins: for example when joining an index (unique) to one or
  more columns in a different `DataFrame`.  
- **many-to-many** joins: joining columns on columns.  


>**Note**
>
>When joining columns on columns (potentially a many-to-many join), any
indexes on the passed `DataFrame` objects **will be discarded**.

It is worth spending some time understanding the result of the **many-to-many**
join case. In SQL / standard relational algebra, if a key combination appears
more than once in both tables, the resulting table will have the **Cartesian
product** of the associated data. Here is a very basic example with one unique
key combination:

In [ ]:
left = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

right = pd.DataFrame(
    {
        "key": ["K0", "K1", "K2", "K3"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)
result = pd.merge(left, right, on="key")

In [ ]:
p.plot([left, right], result, labels=["left","right"], vertical=False);



Here is a more complicated example with multiple join keys. Only the keys
appearing in `left` and `right` are present (the intersection), since
`how='inner'` by default.

In [ ]:
left = pd.DataFrame(
    {
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
    }
)

right = pd.DataFrame(
    {
        "key1": ["K0", "K1", "K1", "K2"],
        "key2": ["K0", "K0", "K0", "K0"],
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
    }
)

result = pd.merge(left, right, on=["key1", "key2"])

In [ ]:
p.plot([left, right], result, labels=["left","right"], vertical=False);


The `how` argument to `merge` specifies how to determine which keys are to
be included in the resulting table. If a key combination **does not appear** in
either the left or right tables, the values in the joined table will be
`NA`. Here is a summary of the `how` options and their SQL equivalent names:

|Merge method|SQL Join Name|Description|
|:------------------|:------------------|:----------------------------------------------------------|
|left|LEFT OUTER JOIN|Use keys from left frame only|
|right|RIGHT OUTER JOIN|Use keys from right frame only|
|outer|FULL OUTER JOIN|Use union of keys from both frames|
|inner|INNER JOIN|Use intersection of keys from both frames|

In [ ]:
result = pd.merge(left, right, how="left", on=["key1", "key2"])
p.plot([left, right], result, labels=["left","right"], vertical=False);

In [ ]:
result = pd.merge(left, right, how="right", on=["key1", "key2"])
p.plot([left, right], result, labels=["left","right"], vertical=False);

In [ ]:
result = pd.merge(left, right, how="outer", on=["key1", "key2"])
p.plot([left, right], result, labels=["left","right"], vertical=False);

In [ ]:
result = pd.merge(left, right, how="inner", on=["key1", "key2"])
p.plot([left, right], result, labels=["left","right"], vertical=False);

In [ ]:
df = pd.DataFrame({"Let": ["A", "B", "C"], "Num": [1, 2, 3]})
df

In [ ]:
ser = pd.Series(
    ["a", "b", "c", "d", "e", "f"],
    index=pd.MultiIndex.from_arrays(
        [["A", "B", "C"] * 2, [1, 2, 3, 4, 5, 6]], names=["Let", "Num"]
    ),
)
ser

In [ ]:
pd.merge(df, ser.reset_index(), on=["Let", "Num"])

Here is another example with duplicate join keys in DataFrames:

In [ ]:
left = pd.DataFrame({"A": [1, 2], "B": [2, 2]})
right = pd.DataFrame({"A": [4, 5, 6], "B": [2, 2, 2]})
result = pd.merge(left, right, on="B", how="outer")

p.plot([left, right], result, labels=["left","right"], vertical=False);

>***Warning:***
>
> Joining / merging on duplicate keys can cause a returned frame that is the multiplication of the row dimensions, which may result in memory overflow. It is the user’ s responsibility to manage duplicate values in keys before joining large DataFrames.


<a id='merging-validation'></a>

### Checking for duplicate keys

Users can use the `validate` argument to automatically check whether there
are unexpected duplicates in their merge keys. Key uniqueness is checked before
merge operations and so should protect against memory overflows. Checking key
uniqueness is also a good way to ensure user data structures are as expected.

In the following example, there are duplicate values of `B` in the right
`DataFrame`. As this is not a one-to-one merge – as specified in the
`validate` argument – an exception will be raised.

In [ ]:
left = pd.DataFrame({"A": [1, 2], "B": [1, 2]})
right = pd.DataFrame({"A": [4, 5, 6], "B": [2, 2, 2]})

``` python

In [53]: result = pd.merge(left, right, on="B", how="outer", validate="one_to_one")
...
MergeError: Merge keys are not unique in right dataset; not a one-to-one merge
```    

If the user is aware of the duplicates in the right `DataFrame` but wants to
ensure there are no duplicates in the left DataFrame, one can use the
`validate='one_to_many'` argument instead, which will not raise an exception.

``` python

pd.merge(left, right, on="B", how="outer", validate="one_to_many")
```


<a id='merging-indicator'></a>

### The merge indicator

`merge()` accepts the argument `indicator`. If `True`, a
Categorical-type column called `_merge` will be added to the output object
that takes on values:

|Observation Origin|_merge value|
|:---------------------------------|:--------------|
|Merge key only in 'left' frame|left_only|
|Merge key only in 'right' frame|right_only|
|Merge key in both frames|both|

In [ ]:
df1 = pd.DataFrame({"col1": [0, 1], "col_left": ["a", "b"]})
df2 = pd.DataFrame({"col1": [1, 2, 2], "col_right": [2, 2, 2]})
pd.merge(df1, df2, on="col1", how="outer", indicator=True)

The `indicator` argument will also accept string arguments, in which case the indicator function will use the value of the passed string as the name for the indicator column.

In [ ]:
pd.merge(df1, df2, on="col1", how="outer", indicator="indicator_column")


<a id='merging-dtypes'></a>

### Merge dtypes

Merging will preserve the dtype of the join keys.

In [ ]:
left = pd.DataFrame({"key": [1], "v1": [10]})
left

In [ ]:
right = pd.DataFrame({"key": [1, 2], "v1": [20, 30]})
right

We are able to preserve the join keys:

In [ ]:
pd.merge(left, right, how="outer")

In [ ]:
pd.merge(left, right, how="outer").dtypes

Of course if you have missing values that are introduced, then the
resulting dtype will be upcast.

In [ ]:
pd.merge(left, right, how="outer", on="key")

In [ ]:
pd.merge(left, right, how="outer", on="key").dtypes

Merging will preserve `category` dtypes of the mergands. See also the section on categoricals.

The left frame.

In [ ]:
from pandas.api.types import CategoricalDtype

X = pd.Series(np.random.choice(["foo", "bar"], size=(10,)))
X = X.astype(CategoricalDtype(categories=["foo", "bar"]))

left = pd.DataFrame(
    {"X": X, "Y": np.random.choice(["one", "two", "three"], size=(10,))}
)
left

In [ ]:
left.dtypes

The right frame.

In [ ]:
right = pd.DataFrame(
    {
        "X": pd.Series(["foo", "bar"], dtype=CategoricalDtype(["foo", "bar"])),
        "Z": [1, 2],
    }
)
right

In [ ]:
right.dtypes

The merged result:

In [ ]:
result = pd.merge(left, right, how="outer")
result

In [ ]:
result.dtypes

>**Note**
>
>The category dtypes must be *exactly* the same, meaning the same categories and the ordered attribute.
Otherwise the result will coerce to the categories’ dtype.

>**Note**
>
>Merging on `category` dtypes that are the same can be quite performant compared to `object` dtype merging.


<a id='merging-join-index'></a>

### Joining on index

`DataFrame.join()` is a convenient method for combining the columns of two
potentially differently-indexed `DataFrames` into a single result
`DataFrame`. Here is a very basic example:

In [ ]:
left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]}, index=["K0", "K1", "K2"]
)

In [ ]:
right = pd.DataFrame(
    {"C": ["C0", "C2", "C3"], "D": ["D0", "D2", "D3"]}, index=["K0", "K2", "K3"]
)

In [ ]:
result = left.join(right)
p.plot([left, right], result, labels=["left","right"], vertical=False);

In [ ]:
result = left.join(right, how="outer")
p.plot([left, right], result, labels=["left","right"], vertical=False);

The same as above, but with `how='inner'`.

In [ ]:
result = left.join(right, how="inner")
p.plot([left, right], result, labels=["left","right"], vertical=False);

The data alignment here is on the indexes (row labels). This same behavior can
be achieved using `merge` plus additional arguments instructing it to use the
indexes:

In [ ]:
result = pd.merge(left, right, left_index=True, right_index=True, how="outer")
p.plot([left, right], result, labels=["left","right"], vertical=False);

In [ ]:
result = pd.merge(left, right, left_index=True, right_index=True, how="inner")
p.plot([left, right], result, labels=["left","right"], vertical=False);

### Joining key columns on an index

`join()` takes an optional `on` argument which may be a column
or multiple column names, which specifies that the passed `DataFrame` is to be
aligned on that column in the `DataFrame`. These two function calls are
completely equivalent:

``` python

left.join(right, on=key_or_keys)
pd.merge(
    left, right, left_on=key_or_keys, right_index=True, how="left", sort=False
)
```

Obviously you can choose whichever form you find more convenient. For
many-to-one joins (where one of the `DataFrame`’s is already indexed by the
join key), using `join` may be more convenient. Here is a simple example:

In [ ]:
left = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "key": ["K0", "K1", "K0", "K1"],
    }
)

right = pd.DataFrame({"C": ["C0", "C1"], "D": ["D0", "D1"]}, index=["K0", "K1"])
result = left.join(right, on="key")


p.plot([left, right], result, labels=["left","right"], vertical=False);

In [ ]:
result = pd.merge(
    left, right, left_on="key", right_index=True, how="left", sort=False
)

p.plot([left, right], result, labels=["left","right"], vertical=False);

<a id='merging-multikey-join'></a>
To join on multiple keys, the passed DataFrame must have a `MultiIndex`:

In [ ]:
left = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "key1": ["K0", "K0", "K1", "K2"],
        "key2": ["K0", "K1", "K0", "K1"],
    }
)

index = pd.MultiIndex.from_tuples(
    [("K0", "K0"), ("K1", "K0"), ("K2", "K0"), ("K2", "K1")]
)
right = pd.DataFrame(
    {"C": ["C0", "C1", "C2", "C3"], "D": ["D0", "D1", "D2", "D3"]}, index=index
)

Now this can be joined by passing the two key column names:

In [ ]:
result = left.join(right, on=["key1", "key2"])

p.plot([left, right], result, labels=["left","right"], vertical=False);

<a id='merging-df-inner-join'></a>
The default for `DataFrame.join` is to perform a left join (essentially a
“VLOOKUP” operation, for Excel users), which uses only the keys found in the
calling DataFrame. Other join types, for example inner join, can be just as
easily performed:

In [ ]:
result = left.join(right, on=["key1", "key2"], how="inner")

p.plot([left, right], result, labels=["left","right"], vertical=False);

As you can see, this drops any rows where there was no match.


<a id='merging-join-on-mi'></a>

### Joining a single Index to a MultiIndex

You can join a singly-indexed `DataFrame` with a level of a MultiIndexed `DataFrame`.
The level will match on the name of the index of the singly-indexed frame against
a level name of the MultiIndexed frame.

In [ ]:
left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]},
    index=pd.Index(["K0", "K1", "K2"], name="key"),
)

index = pd.MultiIndex.from_tuples(
    [("K0", "Y0"), ("K1", "Y1"), ("K2", "Y2"), ("K2", "Y3")],
    names=["key", "Y"],
)
right = pd.DataFrame(
    {"C": ["C0", "C1", "C2", "C3"], "D": ["D0", "D1", "D2", "D3"]},
    index=index,
)

result = left.join(right, how="inner")

p.plot([left, right], result, labels=["left","right"], vertical=False);

This is equivalent but less verbose and more memory efficient / faster than this.

In [ ]:
result = pd.merge(
    left.reset_index(), right.reset_index(), on=["key"], how="inner"
).set_index(["key","Y"])


p.plot([left, right], result, labels=["left","right"], vertical=False);

### Joining with two MultiIndexes

This is supported in a limited way, provided that the index for the right
argument is completely used in the join, and is a subset of the indices in
the left argument, as in this example:

In [ ]:
leftindex = pd.MultiIndex.from_product(
    [list("abc"), list("xy"), [1, 2]], names=["abc", "xy", "num"]
)
left = pd.DataFrame({"v1": range(12)}, index=leftindex)
left

In [ ]:
rightindex = pd.MultiIndex.from_product(
    [list("abc"), list("xy")], names=["abc", "xy"]
)
right = pd.DataFrame({"v2": [100 * i for i in range(1, 7)]}, index=rightindex)
right

In [ ]:
left.join(right, on=["abc", "xy"], how="inner")

If that condition is not satisfied, a join with two multi-indexes can be
done using the following code.

In [ ]:
leftindex = pd.MultiIndex.from_tuples(
    [("K0", "X0"), ("K0", "X1"), ("K1", "X2")], names=["key", "X"]
)
left = pd.DataFrame(
    {"A": ["A0", "A1", "A2"], "B": ["B0", "B1", "B2"]}, index=leftindex
)

rightindex = pd.MultiIndex.from_tuples(
    [("K0", "Y0"), ("K1", "Y1"), ("K2", "Y2"), ("K2", "Y3")], names=["key", "Y"]
)
right = pd.DataFrame(
    {"C": ["C0", "C1", "C2", "C3"], "D": ["D0", "D1", "D2", "D3"]}, index=rightindex
)

result = pd.merge(
    left.reset_index(), right.reset_index(), on=["key"], how="inner"
).set_index(["key", "X", "Y"])

p.plot([left, right], result, labels=["left","right"], vertical=False);

### Merging on a combination of columns and index levels

Strings passed as the `on`, `left_on`, and `right_on` parameters
may refer to either column names or index level names.  This enables merging
`DataFrame` instances on a combination of index levels and columns without
resetting indexes.

In [ ]:
left_index = pd.Index(["K0", "K0", "K1", "K2"], name="key1")

left = pd.DataFrame(
    {
        "A": ["A0", "A1", "A2", "A3"],
        "B": ["B0", "B1", "B2", "B3"],
        "key2": ["K0", "K1", "K0", "K1"],
    },
    index=left_index,
)

right_index = pd.Index(["K0", "K1", "K2", "K2"], name="key1")

right = pd.DataFrame(
    {
        "C": ["C0", "C1", "C2", "C3"],
        "D": ["D0", "D1", "D2", "D3"],
        "key2": ["K0", "K0", "K0", "K1"],
    },
    index=right_index,
)

result = left.merge(right, on=["key1", "key2"])

p.plot([left, right], result, labels=["left","right"], vertical=False);



>**Note**
>
>When DataFrames are merged on a string that matches an index level in both
frames, the index level is preserved as an index level in the resulting
DataFrame.

>**Note**
>
>When DataFrames are merged using only some of the levels of a `MultiIndex`,
the extra levels will be dropped from the resulting merge. In order to
preserve those levels, use `reset_index` on those level names to move
those levels to columns prior to doing the merge.

>**Note**
>
>If a string matches both a column name and an index level name, then a
warning is issued and the column takes precedence. This will result in an
ambiguity error in a future version.

### Overlapping value columns

The merge `suffixes` argument takes a tuple of list of strings to append to
overlapping column names in the input `DataFrame`s to disambiguate the result
columns:

In [ ]:
left = pd.DataFrame({"k": ["K0", "K1", "K2"], "v": [1, 2, 3]})
right = pd.DataFrame({"k": ["K0", "K0", "K3"], "v": [4, 5, 6]})

result = pd.merge(left, right, on="k")


p.plot([left, right], result, labels=["left","right"], vertical=False);

In [ ]:
result = pd.merge(left, right, on="k", suffixes=("_l", "_r"))

p.plot([left, right], result, labels=["left","right"], vertical=False);

`DataFrame.join()` has `lsuffix` and `rsuffix` arguments which behave
similarly.

In [ ]:
left = left.set_index("k")
right = right.set_index("k")
result = left.join(right, lsuffix="_l", rsuffix="_r")

p.plot([left, right], result, labels=["left","right"], vertical=False);

### Joining multiple DataFrames

A list or tuple of `DataFrames` can also be passed to `join()`
to join them together on their indexes.

In [ ]:
right2 = pd.DataFrame({"v": [7, 8, 9]}, index=["K1", "K1", "K2"])
result = left.join([right, right2])

p.plot([left, right], result, labels=["left","right"], vertical=False);

### Merging together values within Series or DataFrame columns

Another fairly common situation is to have two like-indexed (or similarly
indexed) `Series` or `DataFrame` objects and wanting to “patch” values in
one object from values for matching indices in the other. Here is an example:

In [ ]:
df1 = pd.DataFrame(
    [[np.nan, 3.0, 5.0], [-4.6, np.nan, np.nan], [np.nan, 7.0, np.nan]]
)
df2 = pd.DataFrame([[-42.6, np.nan, -8.2], [-5.0, 1.6, 4]], index=[1, 2])

For this, use the `combine_first()` method:

In [ ]:
result = df1.combine_first(df2)
p.plot([df1, df2], result, labels=["df1","df2"], vertical=False);

Note that this method only takes values from the right `DataFrame` if they are
missing in the left `DataFrame`. A related method, `update()`,
alters non-NA values in place:

In [ ]:
df1_copy = df1.copy()


In [ ]:
df1.update(df2)
p.plot([df1, df2], result, labels=["df1","df2"], vertical=False);

## Timeseries friendly merging


<a id='merging-merge-ordered'></a>

### Merging ordered data

A `merge_ordered()` function allows combining time series and other
ordered data. In particular it has an optional `fill_method` keyword to
fill/interpolate missing data:

In [ ]:
left = pd.DataFrame(
    {"k": ["K0", "K1", "K1", "K2"], "lv": [1, 2, 3, 4], "s": ["a", "b", "c", "d"]}
)

right = pd.DataFrame({"k": ["K1", "K2", "K4"], "rv": [1, 2, 3]})

pd.merge_ordered(left, right, fill_method="ffill", left_by="s")


<a id='merging-merge-asof'></a>

### Merging asof

A `merge_asof()` is similar to an ordered left-join except that we match on
nearest key rather than equal keys. For each row in the `left` `DataFrame`,
we select the last row in the `right` `DataFrame` whose `on` key is less
than the left’s key. Both DataFrames must be sorted by the key.

Optionally an asof merge can perform a group-wise merge. This matches the
`by` key equally, in addition to the nearest match on the `on` key.

For example; we might have `trades` and `quotes` and we want to `asof`
merge them.

In [ ]:
trades = pd.DataFrame(
    {
        "time": pd.to_datetime(
            [
                "20160525 13:30:00.023",
                "20160525 13:30:00.038",
                "20160525 13:30:00.048",
                "20160525 13:30:00.048",
                "20160525 13:30:00.048",
            ]
        ),
        "ticker": ["MSFT", "MSFT", "GOOG", "GOOG", "AAPL"],
        "price": [51.95, 51.95, 720.77, 720.92, 98.00],
        "quantity": [75, 155, 100, 100, 100],
    },
    columns=["time", "ticker", "price", "quantity"],
)

quotes = pd.DataFrame(
    {
        "time": pd.to_datetime(
            [
                "20160525 13:30:00.023",
                "20160525 13:30:00.023",
                "20160525 13:30:00.030",
                "20160525 13:30:00.041",
                "20160525 13:30:00.048",
                "20160525 13:30:00.049",
                "20160525 13:30:00.072",
                "20160525 13:30:00.075",
            ]
        ),
        "ticker": ["GOOG", "MSFT", "MSFT", "MSFT", "GOOG", "AAPL", "GOOG", "MSFT"],
        "bid": [720.50, 51.95, 51.97, 51.99, 720.50, 97.99, 720.50, 52.01],
        "ask": [720.93, 51.96, 51.98, 52.00, 720.93, 98.01, 720.88, 52.03],
    },
    columns=["time", "ticker", "bid", "ask"],
)

In [ ]:
trades

In [ ]:
quotes

By default we are taking the asof of the quotes.

In [ ]:
pd.merge_asof(trades, quotes, on="time", by="ticker")

We only asof within `2ms` between the quote time and the trade time.

In [ ]:
pd.merge_asof(trades, quotes, on="time", by="ticker", tolerance=pd.Timedelta("2ms"))

We only asof within `10ms` between the quote time and the trade time and we
exclude exact matches on time. Note that though we exclude the exact matches
(of the quotes), prior quotes **do** propagate to that point in time.

In [ ]:
pd.merge_asof(
    trades,
    quotes,
    on="time",
    by="ticker",
    tolerance=pd.Timedelta("10ms"),
    allow_exact_matches=False,
)


<a id='merging-compare'></a>

## Comparing objects

The `compare()` and `compare()` methods allow you to
compare two DataFrame or Series, respectively, and summarize their differences.

This feature was added in [V1.1.0](../03_whatsnew/v1.1.0.ipynb#whatsnew-110-dataframe-or-series-comparing).

For example, you might want to compare two `DataFrame` and stack their differences
side by side.

In [ ]:
df = pd.DataFrame(
    {
        "col1": ["a", "a", "b", "b", "a"],
        "col2": [1.0, 2.0, 3.0, np.nan, 5.0],
        "col3": [1.0, 2.0, 3.0, 4.0, 5.0],
    },
    columns=["col1", "col2", "col3"],
)
df

In [ ]:
df2 = df.copy()
df2.loc[0, "col1"] = "c"
df2.loc[2, "col3"] = 4.0
df2

In [ ]:
df.compare(df2)

By default, if two corresponding values are equal, they will be shown as `NaN`.
Furthermore, if all values in an entire row / column, the row / column will be
omitted from the result. The remaining differences will be aligned on columns.

If you wish, you may choose to stack the differences on rows.

In [ ]:
df.compare(df2, align_axis=0)

If you wish to keep all original rows and columns, set `keep_shape` argument
to `True`.

In [ ]:
df.compare(df2, keep_shape=True)

You may also keep all the original values even if they are equal.

In [ ]:
df.compare(df2, keep_shape=True, keep_equal=True)